# Calculate Descriptors


This notebook documents the use of transformers to calculate ROBUST descriptors from molecular dynamics simulations. 


### Disclaimer

This notebook is **not** optimized for speed and depending on the size of the dataset can take a very long time to run. A major slowdown is, that all trajectories in this study were stored as tarballs and have to be extracted before descriptors can be calculated.

Several obvious strategies can be employed to speed up computation:

**1. Store uncompressed trajectories**

If storage space is not an issue, uncompressed trajectories can be stored instead. This will eliminate having to copy the tarball to a temporary directory and having to extract the trajectory.

**2. Calculate descriptors simultaneously**

For demonstration purposes descriptors are calculated sequentially. This means that for each descriptor the trajectory tarball has to be copied and extracted. If one would calculate descriptors simultaneously (i.e. in pass through the dataset) the trajectory would only have to be extracted once.  

## Imports

In [4]:
import os
import sys
import shutil
import tempfile

import tarfile

import numpy as np
import pandas as pd

sys.path.append('../../transformers/transformers')
from trj_nonbonded_transformer import run as run_nonbonded
from trj_rms_transformer import run as run_rms
from trj_hbond_transformer import run as run_hbond

## Load dataset

In [23]:
cwd = os.getcwd()

data_dir = os.path.abspath('./data')
dataset_path = os.path.join(data_dir,'dataset.csv')

dataset = pd.read_csv(dataset_path, sep=',', index_col=0)
dataset.rep = dataset.rep.astype(int)

## Check file availability

Check if descriptors have already been calculated.


In [24]:
trj = '/storage/DHFR/PJirovecii/{n}/rep{r}/{n}_rep{r}_14-out.tgz'
desmond_cms = './data/{n}/rep{r}/{n}_rep{r}_14-out.cms'
trj_rms = './data/{n}/rep{r}/{n}_rep{r}_trj_rms.json'
trj_nonbonded = './data/{n}/rep{r}/{n}_rep{r}_trj_nonbonded.json'
trj_hbonds = './data/{n}/rep{r}/{n}_rep{r}_trj_hbond.csv'
com_dist = './data/{n}/rep{r}/{n}_rep{r}_com_dist.csv'

for i in dataset.index:
    n = dataset.loc[i, 'name']
    if not os.path.isdir(os.path.join(data_dir, n)):
        os.mkdir(os.path.join(data_dir, n))
    r = dataset.loc[i, 'rep']
    if not os.path.isdir(os.path.join(data_dir, n, 'rep{}'.format(r))):
        os.mkdir(os.path.join(data_dir, n, 'rep{}'.format(r)))
    trj_arch = trj.format(n=n, r=r)
    if os.path.isfile(trj_arch):
        dataset.loc[i, 'raw_trj'] = trj_arch
    if os.path.isfile(desmond_cms.format(n=n, r=r)):
        dataset.loc[i, 'desmond_cms'] = desmond_cms.format(n=n, r=r)
    if os.path.isfile(trj_rms.format(n=n, r=r)):
        dataset.loc[i, 'trj_rms'] = trj_rms.format(n=n, r=r)
    if os.path.isfile(trj_nonbonded.format(n=n, r=r)):
        dataset.loc[i, 'desmond_nonbonded'] = trj_nonbonded.format(n=n, r=r)
    if os.path.isfile(trj_hbonds.format(n=n, r=r)):
        dataset.loc[i, 'trj_hbonds'] = trj_hbonds.format(n=n, r=r)
    if os.path.isfile(com_dist.format(n=n, r=r)):
        dataset.loc[i, 'com_dist'] = com_dist.format(n=n, r=r)

## Get desmond_cms

In [26]:
for i in dataset.index:
    if not dataset.isna().loc[i, 'raw_trj'] and dataset.isna().loc[i, 'desmond_cms']:
        n = dataset.loc[i, 'name']
        r = int(dataset.loc[i, 'rep'])
        print(n, r)
        desmond_cms = os.path.join(data_dir, '{n}/rep{r}/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
        with tempfile.TemporaryDirectory() as tempdir:
            os.chdir(tempdir)
            shutil.copy(dataset.loc[i, 'raw_trj'], os.path.join(tempdir,'raw_trj.tgz'))
            with tarfile.open('raw_trj.tgz') as tar:
                tar.extractall()
            tmp_cms = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
            shutil.copy(tmp_cms, desmond_cms)
        dataset.loc[i, 'desmond_cms'] = desmond_cms                           

OG4


## Calculate nonbonded descriptors

In [27]:
for i in dataset.index:
    if not dataset.isna().loc[i, 'raw_trj'] and dataset.isna().loc[i, 'desmond_nonbonded']:
        n = dataset.loc[i, 'name']
        r = int(dataset.loc[i, 'rep'])
        print(n, r)
        with tempfile.TemporaryDirectory() as tempdir:
            os.chdir(tempdir)
            shutil.copy(dataset.loc[i, 'raw_trj'], os.path.join(tempdir,'raw_trj.tgz'))
            with tarfile.open('raw_trj.tgz') as tar:
                tar.extractall()
            cmsfile = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
            cfgfile = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cfg'.format(n=n, r=r))
            trjtar = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14_trj'.format(n=n, r=r))
            structure_dict_list = [
                                    {'files': {
                                        'desmond_cms': cmsfile,
                                        'desmond_trjtar': trjtar,
                                        'desmond_cfg': cfgfile
                                    },
                                    'structure': {'code': '{}_rep{}'.format(n,r),
                                    'structure_id': 0},
                                    'custom':[]
                                    }
                                    ]
            for sd in run_nonbonded(structure_dict_list):
                if 'desmond_nonbonded' in sd['files']:
                    nonbonded_raw = os.path.join(data_dir, n, 'rep{}'.format(r))
                    shutil.copy(sd['files']['desmond_nonbonded'], nonbonded_raw)
                    dataset.loc[i, 'desmond_nonbonded'] = os.path.join(nonbonded_raw, sd['files']['desmond_nonbonded'])                         
        os.chdir(cwd)
            
    else:
        continue

OG4


/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/scipy/optimi

## Calculate rms descriptors

In [28]:
for i in dataset.index:
    if not dataset.isna().loc[i, 'raw_trj'] and dataset.isna().loc[i, 'trj_rms']:
        n = dataset.loc[i, 'name']
        r = int(dataset.loc[i, 'rep'])
        print(n, r)
        with tempfile.TemporaryDirectory() as tempdir:
            os.chdir(tempdir)
            shutil.copy(dataset.loc[i, 'raw_trj'], os.path.join(tempdir,'raw_trj.tgz'))
            with tarfile.open('raw_trj.tgz') as tar:
                tar.extractall()
            cmsfile = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
            trjtar = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14_trj'.format(n=n, r=r))
            structure_dict_list = [
                                    {'files': {
                                        'desmond_cms': cmsfile,
                                        'desmond_trjtar': trjtar,
                                    },
                                    'structure': {'code': '{}_rep{}'.format(n,r),
                                    'structure_id': 0},
                                    'custom':[]
                                    }
                                    ]
            for sd in run_rms(structure_dict_list):
                if 'trj_rms' in sd['files']:
                    storage_dir = os.path.join(data_dir, n, 'rep{}'.format(r))
                    shutil.copy(sd['files']['trj_rms'], storage_dir)
                    dataset.loc[i, 'trj_rms'] = os.path.join(storage_dir, sd['files']['trj_rms'])                         
        os.chdir(cwd)
            
    else:
        continue

OG4 3


## Calculate hbond descriptors

In [29]:
for i in dataset.index:
    if not dataset.isna().loc[i, 'raw_trj'] and dataset.isna().loc[i, 'trj_hbonds']:
        n = dataset.loc[i, 'name']
        r = int(dataset.loc[i, 'rep'])
        print(n, r)
        with tempfile.TemporaryDirectory() as tempdir:
            os.chdir(tempdir)
            shutil.copy(dataset.loc[i, 'raw_trj'], os.path.join(tempdir,'raw_trj.tgz'))
            with tarfile.open('raw_trj.tgz') as tar:
                tar.extractall()
            cmsfile = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
            trjtar = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14_trj'.format(n=n, r=r))
            structure_dict_list = [
                                    {'files': {
                                        'desmond_cms': cmsfile,
                                        'desmond_trjtar': trjtar,
                                    },
                                    'structure': {'code': '{}_rep{}'.format(n,r),
                                    'structure_id': 0},
                                    'custom':[]
                                    }
                                    ]
            for sd in run_hbond(structure_dict_list):
                if 'trj_hbonds' in sd['files']:
                    storage_dir = os.path.join(data_dir, n, 'rep{}'.format(r))
                    shutil.copy(sd['files']['trj_hbonds'], storage_dir)
                    dataset.loc[i, 'trj_hbonds'] = os.path.join(storage_dir, sd['files']['trj_hbonds'])                         
        os.chdir(cwd)
            
    else:
        continue

OG4 3


../../transformers/transformers/trj_hbond_transformer.py:202: RuntimeWarning: invalid value encountered in arccos
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_transformer.py:98: RuntimeWarning: overflow encountered in exp
../../transformers/transformers/trj_hbond_tran

## Calculate torsion descriptors

## Calculate distance between protein and ligand COM

In [16]:
from schrodinger.application.desmond.packages import topo, traj, analysis

In [ ]:
protein_asl = 'protein and not a.element H'
ligand_asl = 'r.ptype TOP and not a.element H'
for i in dataset.index:
    if not dataset.isna().loc[i, 'raw_trj'] and dataset.isna().loc[i, 'com_dist']:
        n = dataset.loc[i, 'name']
        r = int(dataset.loc[i, 'rep'])
        print(n, r)
        storage_dir = os.path.join(data_dir, n, 'rep{}'.format(r))
        outfile = '{}_rep{}_com_dist.csv'.format(n, r)
        with tempfile.TemporaryDirectory() as tempdir:
            os.chdir(tempdir)
            shutil.copy(dataset.loc[i, 'raw_trj'], os.path.join(tempdir,'raw_trj.tgz'))
            with tarfile.open('raw_trj.tgz') as tar:
                tar.extractall()
            cmsfile = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14-out.cms'.format(n=n, r=r))
            trjdir = os.path.join(tempdir,'{n}_rep{r}_14/{n}_rep{r}_14_trj'.format(n=n, r=r))
            msys_model, cms_model = topo.read_cms(cmsfile)
            tr = traj.read_traj(trjdir)
            protein_com = analysis.Com(msys_model, cms_model, asl=protein_asl)
            ligand_com = analysis.Com(msys_model, cms_model, asl=ligand_asl)
            dist = analysis.Distance(msys_model, cms_model, protein_com, ligand_com)
            results = np.array(analysis.analyze(tr, dist))
            np.savetxt(outfile, results, delimiter=',')
            shutil.copy(outfile, storage_dir)
        dataset.loc[i, 'com_dist'] = os.path.join(storage_dir, outfile)  

## Save changes

In [31]:
dataset.to_csv(dataset_path, sep=',')

## Create feature matrix

Once descriptors have been calculated for all variants we have to combine them in a single feature matrix.
For this purpose I have written a shell script, that sequentialy callse process.py and merge.py, generating a feature matrix for each descriptor.

In [32]:
%%bash
/bin/bash create_feature_matrix.sh ./data/data

Note: The Schrodinger virtualenv is tied to a specific SCHRODINGER value.
This virtualenv is tied to SCHRODINGER=/opt/usershared/software/schrodinger19.1.

If you change your SCHRODINGER environment variable, it will break the ability
to use the unadorned python command.

calculating vdw
calculating elec
calculating rms
calculating hbond
merging vdw
merging elec
merging rms
merging hbond


/home/leidnerf/schrodinger.ve/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
Traceback (most recent call last):
  File "../../transformers/utils/merge.py", line 527, in <module>
    raise ValueError('Merging data files requires either the dataset or a pldb api endpoint')
ValueError: Merging data files requires either the dataset or a pldb api endpoint
Traceback (most recent call last):
  File "../../transformers/utils/merge.py", line 527, in <module>
    raise ValueError('Merging data files requires either the dataset or a pldb api endpoint')
ValueError: Merging data files requires either the dataset or a pldb api endpoint
Traceback (most recent call last):
  File "../../transformers/utils

CalledProcessError: Command 'b'/bin/bash create_feature_matrix.sh ./data/data\n'' returned non-zero exit status 1.